Pre-Analysis Data Considerations: Yes there were missing values. The way that I dealt with them depended on each faeture. I will go into further detail about how I addressed modifying my features in the markdown below.

In [101]:
#Project 2 Nick Hageman
import csv
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import seaborn as sns
from sklearn import tree
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn import (datasets, neighbors,
                     naive_bayes,
                     model_selection as skms,
                     linear_model, dummy,
                     metrics,
                     pipeline,
                     preprocessing as skpre)

Feature Engineering pt1: The below cell are the chosen features for the model. I tried to choose only the features that would be good in differentiating the vehicles in relation to it's price to improve it's predictions. I'll explain what I did to change the NAN data or strings in the next markdown where I modified all the data for the model.

In [123]:

#Choosing features
features = [
    "year",
    "cylinders",
    "condition",
    "odometer",
    "title_status",
    "price"
]

features2 = [
    "year",
    "cylinders",
    "condition",
    "odometer",
    "title_status",
]

Feature Engineering pt2: To begin, I modified most of the features that included strings by replacing each value with a numerical value that I thought would give the features a sense of hierarchy in value. For example, for condition, I replaced the strings of the best condition with the highest numerical value. I did something similar with cylinders, giving the highest number of cylinders the highest numerical value. For title status, I did the same thing as I did with the condition. Now that all of these features had numerical values, I filled the NAN values with their respective means. For the rows in the odometer and year that contained NAN values, I found that I got the best RMSE when I just dropped those rows altogether rather than replacing it with the mean. Finally, I did one-hot encoding for the type of car, I chose this feature because I thought it would have a big impact and it turned out to improve my score significantly.

Another thing to note is that I had the "data_train_drop" data frame so when I dropped the odometer and year rows with NAN values, it would also drop the corresponding price so the data didn't get all discombobulated.

In [ ]:

data_train_df = pd.read_csv("cars_reg_train.csv") 
#Make this temporary feature dataframe that INCLUDES the target so when I drop a row, it also drops the corresponding features to conserve the sizes of the final dataframes
data_train_drop = data_train_df[features]

# Modifying any rows with missing/string values
data_train_drop["condition"] = data_train_drop["condition"].replace("new", 5).replace("like new", 4).replace("excellent", 3).replace("good", 2).replace("fair", 1).replace("salvage", 0).astype(float)
data_train_drop["condition"].fillna(data_train_drop["condition"].mean(), inplace = True)
data_train_drop["cylinders"] = data_train_drop["cylinders"].replace("12 cylinders", 12).replace("10 cylinders", 10).replace("8 cylinders", 8).replace("6 cylinders", 6).replace("5 cylinders", 5).replace("4 cylinders", 4).replace("3 cylinders", 3).replace("other", 0).astype(float)
data_train_drop["cylinders"].fillna(data_train_drop["cylinders"].mean(), inplace = True)
data_train_drop["title_status"] = data_train_drop["title_status"].replace("clean", 3).replace("rebuilt", 2).replace("salvage", 1).replace("missing", 0).replace("parts only", 0).replace("lien", 0).astype(float)
data_train_drop["title_status"].fillna(data_train_drop["title_status"].mean(), inplace = True)

#onehot function to create one-hot dataframes
def onehot(x,str):
    onehot = x[['Unnamed: 0',str]]
    encoder = OneHotEncoder(handle_unknown='ignore')
    encoder_df = pd.DataFrame(encoder.fit_transform(onehot[[str]]).toarray())
    final_df = onehot.join(encoder_df)
    final_df.drop('Unnamed: 0', axis=1, inplace=True)
    final_df.drop(str, axis=1, inplace=True)
    return(final_df)

# working the onehot function for each column
kind = onehot(data_train_df, 'type')
#Assigning values so I don't have to write it all out again
f = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13"]
kind.columns = f

#Remove all rows with NaN values
data_train_drop.dropna(inplace=True)
data_train_df = data_train_drop.copy()

#converts to float16 types
for feature in features:
    if feature != 'price':
        data_train_df[feature] = data_train_df[feature].convert_dtypes(np.float16)
        data_train_df[feature] = data_train_df[feature].convert_dtypes(np.float16)

data_train_ft = pd.DataFrame(data_train_df[features2])
#Adding each column back to the original df
for col in f:
    data_train_ft[col] = kind[col]
data_train_tgt = data_train_df["price"]
display(data_train_ft)

Here I did a training/test split to get different data to 1) find the best model and 2) use the other set to fit the model. I tried to test linear regression and several different regression models including KNN, Lasso, and Ridge. I ended up commenting out KNN because it took a long time and was never being chosen as the best model anyway. The model with the lowest RMSE would be chosen as the best model and would be used to fit the data.

In [ ]:
#----------------------------------
# Split into training/testing data 
#----------------------------------
(data_train_plus_validation_ftrs, 
 data_test_ftrs,
 data_train_plus_validation_tgt, 
 data_test_tgt) = skms.train_test_split(data_train_ft,
                                   data_train_tgt,
                                   test_size=.20)

# separate training/validation sets
(train_ftrs,
 validation_ftrs,
 train_tgt,
 validation_tgt) = skms.train_test_split(data_train_plus_validation_ftrs,
                                         data_train_plus_validation_tgt,
                                         test_size=.25)

# define dictionary of models to try
models_to_try = {'lr': linear_model.LinearRegression()}
# add k-NN models with various values of k to models_to_try
"""for k in range(1,16,2):
    models_to_try[f'{k}-NN'] = neighbors.KNeighborsRegressor(n_neighbors=k)
"""
#LASSO
values = [0.25, 0.50, 0.75, 1.0, 1.25, 1.50, 1.75, 2.0, 3.0, 4.0, 5.0, 10, 20]
for alpha_value in values:
    models_to_try[f'Lasso (C={alpha_value})'] = linear_model.Lasso(alpha=alpha_value)
#RIDGE
alpha_value = 0.1
for alpha_value in values:
    models_to_try[f'ridge (C={alpha_value})'] = linear_model.Ridge(alpha=alpha_value)

#TEST PREDICTION
model_rmse = {}
for model_name in models_to_try:
    fit = models_to_try[model_name].fit(train_ftrs, train_tgt)
    preds  = models_to_try[model_name].predict(validation_ftrs)
    rmse = np.sqrt(metrics.mean_squared_error(validation_tgt, preds))
    model_rmse[model_name] = rmse
    print(f'{model_name} RMSE: {rmse:.4f}')

# get model with lowest error
best_model_name = min(model_rmse,key=model_rmse.get)
print(f'\nBest model: {best_model_name}; RMSE: {model_rmse[best_model_name]:.2f}')
best_model = models_to_try[best_model_name]


Models: Here I just modified the test data to be able to make predictions with my model just like how I modified the features earlier. I then fit the data on the model, made my predictions, and recorded them on a csv file to submit. I attempted to standardize the data but it did not result in a better score so I got rid of it. Cross validation was not used. Another thing to note is that I was not able to drop any NAN values because I needed to make predictions on ALL the prices so I just replaced them with the mean.

In [ ]:

# This is just using the test.csv to setup a dataframe of the correct size
# and indicies (the "id" field).
make_submission_df = pd.read_csv("cars_reg_test.csv")
submission_ftrs = make_submission_df[features2]

#Modify testing data
#Remove all rows with NaN values
submission_ftrs["condition"] = submission_ftrs["condition"].replace("new", 5).replace("like new", 4).replace("excellent", 3).replace("good", 2).replace("fair", 1).replace("salvage", 0).astype(float)
submission_ftrs["condition"].fillna(submission_ftrs["condition"].mean(), inplace = True)
submission_ftrs["cylinders"] = submission_ftrs["cylinders"].replace("12 cylinders", 12).replace("10 cylinders", 10).replace("8 cylinders", 8).replace("6 cylinders", 6).replace("5 cylinders", 5).replace("4 cylinders", 4).replace("3 cylinders", 3).replace("other", 0).astype(float)
submission_ftrs["cylinders"].fillna(submission_ftrs["cylinders"].mean(), inplace = True)
submission_ftrs["title_status"] = submission_ftrs["title_status"].replace("clean", 3).replace("rebuilt", 2).replace("salvage", 1).replace("missing", 0).replace("parts only", 0).replace("lien", 0).astype(float)
submission_ftrs["title_status"].fillna(submission_ftrs["title_status"].mean(), inplace = True)
submission_ftrs["year"].fillna(submission_ftrs["year"].mean(), inplace = True)
submission_ftrs["odometer"].fillna(submission_ftrs["odometer"].mean(), inplace = True)

#onehot function to create one-hot dataframes
def onehot(x,str):
    onehot = x[['id',str]]
    encoder = OneHotEncoder(handle_unknown='ignore')
    encoder_df = pd.DataFrame(encoder.fit_transform(onehot[[str]]).toarray())
    final_df = onehot.join(encoder_df)
    final_df.drop('id', axis=1, inplace=True)
    final_df.drop(str, axis=1, inplace=True)
    return(final_df)
''''''
# working the onehot function for each column
kind = onehot(make_submission_df, 'type')
#Assigning values so I don't have to write it all out again
f = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13"]
kind.columns = f
#Adding each column back to the original df
for col in f:
    submission_ftrs[col] = kind[col]
#submission_ftrs.drop("type", axis=1, inplace=True)
display(submission_ftrs)

#converts to float16 types
for feature in features:
    if feature != 'price':
        submission_ftrs[feature] = submission_ftrs[feature].convert_dtypes(np.float16)
        submission_ftrs[feature] = submission_ftrs[feature].convert_dtypes(np.float16)

# drop all columns except 'id'
make_submission_df = make_submission_df[['id']]
# make sure the column of ID's that we just read in is the index column
make_submission_df = make_submission_df.set_index('id')

fit = best_model.fit(data_test_ftrs, data_test_tgt)
predictions = best_model.predict(submission_ftrs)

# Here, you add your predictions to the dataframe
make_submission_df['price'] = predictions

# Either one of these will work
# The first one will round all floating point numbers to 2 decimals
# Makes it easier to look at.
make_submission_df.to_csv('submission.csv',sep=',', float_format='%.2f')
#make_submission_df.to_csv('submission.csv',sep=',')

